In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1,6,5)
        self.conv2 = nn.Conv2d(6,16,5)
        # an affine operation
        self.fc1 = nn.Linear(16*5*5,120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)

    def forward(self,x):
        # Max Pooling (2,2)
        x = F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        # if the size is a square, you can specify with a single number
        x = F.max_pool2d(F.relu(self.conv2(x)),(2))
        x = torch.flatten(x,1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [3]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [4]:
params = list(net.parameters())
print(len(params))
print(params[0].size()) # conv1의 weight

10
torch.Size([6, 1, 5, 5])


In [5]:
input = torch.randn(1,1,32,32)
out= net(input)
print(out)

tensor([[ 0.0756,  0.0657, -0.0588, -0.0079,  0.1470, -0.1071,  0.0752, -0.0323,
         -0.1105,  0.1041]], grad_fn=<AddmmBackward0>)


In [6]:
net.zero_grad()
out.backward(torch.randn(1,10))

### Loss Fucntion

In [7]:
output = net(input)
target = torch.randn(10)
target = target.view(1,-1)
criterion = nn.MSELoss()

loss = criterion(output,target)
print(loss)

tensor(0.9215, grad_fn=<MseLossBackward0>)


### loss.backward()

In [10]:
print(loss.grad_fn) #MSELoss
print(loss.grad_fn.next_functions[0][0]) # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0]) # ReLU

### BackProp

In [11]:
net.zero_grad()
print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()
print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0059, -0.0026, -0.0036, -0.0070, -0.0133,  0.0144])


### Update the weights

In [12]:
import torch.optim as optim

In [13]:
optimizer = optim.SGD(net.parameters(),lr=0.01)

optimizer.zero_grad() # gradient buffer 0으로 초기화
ouput = net(input)
loss = criterion(output,target) # mse loss
loss.backward()
optimizer.step() # update

RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.